# BBC Location ID Weather Scraping

In [1]:
import os

import requests                          # to get the html code for the webpage
import json                              # to convert API to json format

from urllib.parse import urlencode

import numpy as np
import pandas as pd
import re                                # regular expression operators

In [ ]:
test_city = 'Toronto'

location_url = 'https://locator-service.api.bbci.co.uk/locations?' + urlencode({
    'api_key': 'AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv',
    's': test_city,
    'stack': 'aws',
    'locale': 'en',
    'filter': 'international',
    'place-types': 'settlement,airport,district',
    'order': 'importance',
    'a': 'true',
    'format': 'json'
})

location_url

'https://locator-service.api.bbci.co.uk/locations?api_key=AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv&s=Toronto&stack=aws&locale=en&filter=international&place-types=settlement%2Cairport%2Cdistrict&order=importance&a=true&format=json'

  
Got the location api from: inspect( right click on web page ) --> network

In [ ]:
result = requests.get(location_url).json()
result

{'response': {'results': {'results': [{'id': '6167865',
     'name': 'Toronto',
     'container': 'Canada',
     'containerId': 6251999,
     'language': 'en',
     'timezone': 'America/Toronto',
     'country': 'CA',
     'latitude': 43.70011,
     'longitude': -79.4163,
     'placeType': 'settlement'},
    {'id': '4251360',
     'name': 'Toronto',
     'container': 'United States',
     'containerId': 6252001,
     'language': 'en',
     'timezone': 'America/Chicago',
     'country': 'US',
     'latitude': 39.71394,
     'longitude': -89.62982,
     'placeType': 'settlement'},
    {'id': '6296338',
     'name': 'Toronto Pearson International Airport',
     'container': 'Canada',
     'containerId': 6251999,
     'language': 'en',
     'timezone': 'America/Toronto',
     'country': 'CA',
     'latitude': 43.68066,
     'longitude': -79.61286,
     'placeType': 'airport'}],
   'totalResults': 3}}}

In [ ]:
loc_id = result['response']['results']['results'][0]['id']
loc_id

'6167865'

#### Creating a function to get id

In [2]:
def get_id(city):
    city = city.lower()
    location_url = 'https://locator-service.api.bbci.co.uk/locations?' + urlencode({
    'api_key': 'AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv',
    's': city,
    'stack': 'aws',
    'locale': 'en',
    'filter': 'international',
    'place-types': 'settlement,airport,district',
    'order': 'importance',
    'a': 'true',
    'format': 'json'
                       })
    
    result = requests.get(location_url).json()

    loc_id = result['response']['results']['results'][0]['id']

    return loc_id

In [ ]:
get_id('new York')

'5128581'

In [ ]:
get_id('mumbai')

'1275339'

In [3]:
get_id('Fort Worth')

'4691930'

# BBC Weather Scraping

In [ ]:
import json                              # to convert API to json format

from urllib.parse import urlencode

import requests                          # to get the web page
from bs4 import BeautifulSoup            # to parse the web page

import pandas as pd
import re

from datetime import datetime

In [ ]:
# eg. url for mumbai --> https://www.bbc.com/weather/1275339

city_name = 'Mumbai'
city_id = get_id(city_name)                  # function defined in prev module

url = 'https://www.bbc.com/weather/' + city_id

response = requests.get(url)

In [ ]:
soup = BeautifulSoup( response.content, 'html.parser')

#### The information we want ( daily high and low temp. and daily weather summary ), are in specific blocks on the webpage. We need to find the block type, type of identifier and identifier name. ( all these can be figured out by right clicking on the web page and selecting **'inspect'** on the Chrome.

In [ ]:
daily_high_value = soup.find_all('span', attrs = {'class': 'wr-day-temperature__high-value'})

# block-type: span , identifier type: class,  class name = 'wr-day-temperature__high-value' 

daily_high_value        # display values for 14 days

[<span class="wr-day-temperature__high-value"><span class="wr-value--temperature "><span class="wr-value--temperature--c">27°</span><span class="wr-hide"> </span><span class="wr-value--temperature--f">81°</span></span></span>,
 <span class="wr-day-temperature__high-value"><span class="wr-value--temperature "><span class="wr-value--temperature--c">29°</span><span class="wr-hide"> </span><span class="wr-value--temperature--f">85°</span></span></span>,
 <span class="wr-day-temperature__high-value"><span class="wr-value--temperature "><span class="wr-value--temperature--c">28°</span><span class="wr-hide"> </span><span class="wr-value--temperature--f">83°</span></span></span>,
 <span class="wr-day-temperature__high-value"><span class="wr-value--temperature "><span class="wr-value--temperature--c">28°</span><span class="wr-hide"> </span><span class="wr-value--temperature--f">83°</span></span></span>,
 <span class="wr-day-temperature__high-value"><span class="wr-value--temperature "><span cla

In [ ]:
daily_low_value = soup.find_all('span' , attrs = {'class': 'wr-day-temperature__low-value'})

In [ ]:
daily_summary = soup.find('div' , attrs = {'class': 'wr-day-summary'})
daily_summary

<div class="wr-day-summary"><div class="gel-wrap"><span class="">Thundery showers and a gentle breeze</span><span class="wr-hide">Thundery showers and a moderate breeze</span><span class="wr-hide">Thundery showers and a moderate breeze</span><span class="wr-hide">Thundery showers and a gentle breeze</span><span class="wr-hide">Thundery showers and a gentle breeze</span><span class="wr-hide">Light cloud and a gentle breeze</span><span class="wr-hide">Light cloud and a gentle breeze</span><span class="wr-hide">Light cloud and a gentle breeze</span><span class="wr-hide">Light cloud and a gentle breeze</span><span class="wr-hide">Heavy rain and a gentle breeze</span><span class="wr-hide">Thundery showers and a gentle breeze</span><span class="wr-hide">Thundery showers and a gentle breeze</span><span class="wr-hide">Light cloud and a gentle breeze</span><span class="wr-hide">Light cloud and a gentle breeze</span></div></div>

In [ ]:
daily_summary.text

'Thundery showers and a gentle breezeThundery showers and a moderate breezeThundery showers and a moderate breezeThundery showers and a gentle breezeThundery showers and a gentle breezeLight cloud and a gentle breezeLight cloud and a gentle breezeLight cloud and a gentle breezeLight cloud and a gentle breezeHeavy rain and a gentle breezeThundery showers and a gentle breezeThundery showers and a gentle breezeLight cloud and a gentle breezeLight cloud and a gentle breeze'

In [ ]:
daily_high_value[0].text.strip()

'27° 81°'

In [ ]:
daily_high_value[5].text.strip()

'31° 87°'

In [ ]:
daily_high_value[0].text.strip().split()

['27°', '81°']

In [ ]:
daily_high_value[0].text.strip().split()[0]

'27°'

In [ ]:
daily_high_value_list = [daily_high_value[i].text.strip().split()[0] for i in range(len(daily_high_value))]
daily_high_value_list

['27°',
 '29°',
 '28°',
 '28°',
 '29°',
 '31°',
 '31°',
 '31°',
 '31°',
 '30°',
 '29°',
 '29°',
 '30°',
 '31°']

In [ ]:
daily_low_value_list = [daily_low_value[i].text.strip().split()[0] for i in range(len(daily_low_value))]

In [ ]:
daily_summary.text

'Thundery showers and a gentle breezeThundery showers and a moderate breezeThundery showers and a moderate breezeThundery showers and a gentle breezeThundery showers and a gentle breezeLight cloud and a gentle breezeLight cloud and a gentle breezeLight cloud and a gentle breezeLight cloud and a gentle breezeHeavy rain and a gentle breezeThundery showers and a gentle breezeThundery showers and a gentle breezeLight cloud and a gentle breezeLight cloud and a gentle breeze'

In [ ]:
daily_summary_list = re.findall('[a-zA-Z][^A-Z]*' , daily_summary.text) # split the string on uppercase
daily_summary_list

['Thundery showers and a gentle breeze',
 'Thundery showers and a moderate breeze',
 'Thundery showers and a moderate breeze',
 'Thundery showers and a gentle breeze',
 'Thundery showers and a gentle breeze',
 'Light cloud and a gentle breeze',
 'Light cloud and a gentle breeze',
 'Light cloud and a gentle breeze',
 'Light cloud and a gentle breeze',
 'Heavy rain and a gentle breeze',
 'Thundery showers and a gentle breeze',
 'Thundery showers and a gentle breeze',
 'Light cloud and a gentle breeze',
 'Light cloud and a gentle breeze']

#### Now our data is ready. We just need to put it in a dataframe

In [ ]:
datelist = pd.date_range(datetime.today(), periods = len(daily_high_value)).tolist()
datelist

[Timestamp('2022-09-13 10:34:37.709903', freq='D'),
 Timestamp('2022-09-14 10:34:37.709903', freq='D'),
 Timestamp('2022-09-15 10:34:37.709903', freq='D'),
 Timestamp('2022-09-16 10:34:37.709903', freq='D'),
 Timestamp('2022-09-17 10:34:37.709903', freq='D'),
 Timestamp('2022-09-18 10:34:37.709903', freq='D'),
 Timestamp('2022-09-19 10:34:37.709903', freq='D'),
 Timestamp('2022-09-20 10:34:37.709903', freq='D'),
 Timestamp('2022-09-21 10:34:37.709903', freq='D'),
 Timestamp('2022-09-22 10:34:37.709903', freq='D'),
 Timestamp('2022-09-23 10:34:37.709903', freq='D'),
 Timestamp('2022-09-24 10:34:37.709903', freq='D'),
 Timestamp('2022-09-25 10:34:37.709903', freq='D'),
 Timestamp('2022-09-26 10:34:37.709903', freq='D')]

In [ ]:
datelist[0].date()

datetime.date(2022, 9, 13)

In [ ]:
datelist = [datelist[i].date().strftime('%y-%m-%d') for i in range(len(datelist))]
datelist

['22-09-13',
 '22-09-14',
 '22-09-15',
 '22-09-16',
 '22-09-17',
 '22-09-18',
 '22-09-19',
 '22-09-20',
 '22-09-21',
 '22-09-22',
 '22-09-23',
 '22-09-24',
 '22-09-25',
 '22-09-26']

In [ ]:
zipped = zip(datelist, daily_high_value_list, daily_low_value_list, daily_summary_list)
zipped

In [ ]:
df = pd.DataFrame(list(zipped), columns = ['Date', 'High', 'Low', 'Summary'])
df.head()

,Date,High,Low,Summary
0,22-09-13,27°,25°,Thundery showers and a gentle breeze
1,22-09-14,29°,25°,Thundery showers and a moderate breeze
2,22-09-15,28°,25°,Thundery showers and a moderate breeze
3,22-09-16,28°,25°,Thundery showers and a gentle breeze
4,22-09-17,29°,25°,Thundery showers and a gentle breeze


In [ ]:
df['High'] = df['High'].replace('°', "", regex = True ).astype(np.float64)  # remove degree symbol

In [ ]:
df['Low'] = df['Low'].replace('°', "", regex = True ).astype(np.float64)

In [ ]:
df.head(3)

,Date,High,Low,Summary
0,22-09-13,27.0,25.0,Thundery showers and a gentle breeze
1,22-09-14,29.0,25.0,Thundery showers and a moderate breeze
2,22-09-15,28.0,25.0,Thundery showers and a moderate breeze


#### Extract the name of city for which data is gathered

In [ ]:
location = soup.find('h1' , attrs =  {'id': 'wr-location-name-id'})
location.text.split()

['Mumbai', '-', 'Weather', 'warnings', 'issued']

#### Saving file in csv format

In [ ]:
filename_csv = location.text.split()[0] + ".csv"
df.to_csv(filename_csv , index = None)

# IMDb web scraping

Our goal is to convert the top 250 list of movies in IMDb into a tabular form using python.

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [ ]:
r = requests.get('https://www.imdb.com/chart/top/')

# convert to a beautiful soup object
soup = bs(r.content)

# Print out HTML
contents = soup.prettify()
print(contents)

In [ ]:
movie_title = []
movie_year = []
movie_rating = []


#### Extract HTML tag contents

In [ ]:
imdb_table = soup.find(class_ = 'chart full-width')

In [ ]:
# Now we will search under imdb_table

movie_titlecolumn = imdb_table.find_all( class_ = 'titleColumn')

In [ ]:
movie_ratingcolumn = imdb_table.find_all( class_ = 'ratingColumn imdbRating')

In [ ]:
for row in movie_titlecolumn:
    title = row.a.text          # 'a' represent tag as title is contained under <a href > tag
    movie_title.append(title)

movie_title[:5]

['The Shawshank Redemption',
 'The Godfather',
 'The Dark Knight',
 'The Godfather Part II',
 '12 Angry Men']

In [ ]:
for row in movie_titlecolumn:
    year = row.span.text                  # span represents tag under class = 'titleColumn'
    movie_year.append(year)

movie_year[:5]

['(1994)', '(1972)', '(2008)', '(1974)', '(1957)']

In [ ]:
for row in movie_ratingcolumn:
    rating = row.strong.text              # strong represents tag under class = 'ratingColumn imdbRating'
    movie_rating.append(rating)

movie_rating[:5]

['9.2', '9.2', '9.0', '9.0', '8.9']

#### Creating a DataFrame

In [ ]:
movie_df = pd.DataFrame({'Movie Title': movie_title,
                         'Year of Release' : movie_year,
                         'IMDb Rating' : movie_rating})

In [ ]:
movie_df.head()

,Movie Title,Year of Release,IMDb Rating
0,The Shawshank Redemption,(1994),9.2
1,The Godfather,(1972),9.2
2,The Dark Knight,(2008),9.0
3,The Godfather Part II,(1974),9.0
4,12 Angry Men,(1957),8.9


In [ ]:
movie_df['Year of Release'] = movie_df['Year of Release'].str.strip('()').astype(int)

In [ ]:
movie_df['IMDb Rating'] = movie_df['IMDb Rating'].astype(float)

In [ ]:
movie_df.head()

,Movie Title,Year of Release,IMDb Rating
0,The Shawshank Redemption,1994,9.2
1,The Godfather,1972,9.2
2,The Dark Knight,2008,9.0
3,The Godfather Part II,1974,9.0
4,12 Angry Men,1957,8.9
